# Night weather synopsis for {{ params.dayobs }}

This notebook shows how a synopsis of weather data over a night can be compiled by querying the EFD, particularly the `lsst.sal.ESS` (Environmental Sensor Suite) and `lsst.sal.DIMM` (seeing monitor) topics.

In [ ]:
# Times Square parameters
dayobs = '20240114'

In [ ]:
from datetime import datetime, timedelta, timezone, UTC
from zoneinfo import ZoneInfo

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from astroplan import Observer
from astropy.time import Time
from IPython.display import Markdown
from lsst_efd_client import EfdClient

In [ ]:
# Establish the timeframe for sun set/rise for the night

local_timezone = ZoneInfo("America/Santiago")
timestamp_format = "%Y-%m-%d %H:%M:%S %Z"
auxtel_site = Observer.at_site("Rubin AuxTel")

# Parse dayobs into a date with UTC-12 timezone
dayobs_start = datetime.strptime(dayobs, '%Y%m%d').replace(
    hour=0, minute=0, second=0, tzinfo=timezone(-timedelta(hours=12))
)

# Datetimes to establish reference points for finding sunset times
# and measurement times for moon illumination
noon_local_datetime = dayobs_start.astimezone(local_timezone).replace(
    hour=12, minute=0, second=0, tzinfo=local_timezone
)
midnight_local_datetime = dayobs_start.astimezone(local_timezone).replace(
    hour=23, minute=59, second=59, tzinfo=local_timezone
)
# Astropy Time for astroplan calculations
noon_time = Time(noon_local_datetime, scale="utc")
midnight_time = Time(midnight_local_datetime, scale="utc")

# Compute sunrise/sunset
sunset_time = auxtel_site.sun_set_time(
    noon_time,
    which="next"
)
sunset_datetime = sunset_time.to_datetime(timezone=UTC)
sunrise_time = auxtel_site.sun_rise_time(
    noon_time,
    which="next"
)
sunrise_datetime = sunrise_time.to_datetime(timezone=UTC)

# Nautical sunset and sunrise (preferred by observers over astronomical twilight)
evening_twilight_datetime = auxtel_site.twilight_evening_nautical(
    noon_time,
    which="next"
).to_datetime(timezone=UTC)
morning_twilight_datetime = auxtel_site.twilight_morning_nautical(
    noon_time,
    which="next"
).to_datetime(timezone=UTC)

In [ ]:
# Warn if data is incomplete
now = datetime.now(tz=UTC)
if now < sunset_datetime:
    print("This report is for the future. Data is unavailable.")
elif now < sunrise_datetime:
    print("This report was computed before sunrise. Data may be incomplete.")

In [ ]:
efd_client = EfdClient("usdf_efd")

async def query_night(topic, fields, index=None):
    """Query an EFD topic and field(s) from sunset to sunrise."""
    # note that sunset/sunrise_time as set from global state
    # outside this function
    return await efd_client.select_time_series(
        topic,
        fields,
        sunset_time,
        sunrise_time,
        index=index
    )

In [ ]:
# Data queries
ess_temp_df = await query_night("lsst.sal.ESS.temperature", "temperatureItem0", index=301) # C
ess_dewpoint_df = await query_night("lsst.sal.ESS.dewPoint", "dewPointItem", index=301) # C
ess_humidity_df = await query_night("lsst.sal.ESS.relativeHumidity", "relativeHumidityItem", index=301) # %
ess_pressure_df = await query_night("lsst.sal.ESS.pressure", "pressureItem0", index=301) # mbar
ess_wind_df = await query_night("lsst.sal.ESS.airFlow", ["speed", "direction"], index=301) # m/s
dimm_fwhm_df = await query_night("lsst.sal.DIMM.logevent_dimmMeasurement", "fwhm") # arcsec

In [ ]:
def conf_axes(ax, label_xaxis=False, label_twilight=False):
    """Configure axis for plotting a night.

    Functionality includes:

    - plotting the twilight period (and optionally labelling it)
    - setting the axis limits
    - hiding the x-axis labels (optionally) as needed in the axes grid.
    """
    # Note: this function depends on global state for the sunset/sunrise
    # and twilight datetimes.
    ax.set_xlim([sunset_datetime, sunrise_datetime])
    ax.axvspan(sunset_datetime, evening_twilight_datetime, fc='r', alpha=0.2, zorder=-2)
    ax.axvspan(morning_twilight_datetime, sunrise_datetime, fc='r', alpha=0.2, zorder=-2)
    if not label_xaxis:
        ax.axes.xaxis.set_ticklabels([])

    if label_twilight:
        ax.text(
            evening_twilight_datetime - timedelta(minutes=30),
            0.5,
            "twilight",
            c='r',
            size=15,
            rotation=90,
            transform=ax.get_xaxis_transform(),
            horizontalalignment='center',
            verticalalignment='center',
            zorder=-1
        )
        ax.text(
            morning_twilight_datetime + timedelta(minutes=30),
            0.5,
            "twilight",
            c='r',
            size=15,
            rotation=90,
            transform=ax.get_xaxis_transform(),
            horizontalalignment='center',
            verticalalignment='center',
            zorder=-1
        )


def draw_summary_table(ax, df, df_name, label, units_label):
    """Draw a table with summary statistics."""
    seeing_q = df.quantile([0.05, 0.5, 0.95])[df_name]
    table = ax.table(
        [
            [label, units_label],
            ["Min (5%)", f"{seeing_q[0.05]:.1f}"],
            ["Median", f"{seeing_q[0.5]:.1f}"],
            ["Max (95%)", f"{seeing_q[0.95]:.1f}"]
        ],
        loc="right",
        colWidths=[0.2, 0.1]
    )
    table.scale(1, 1.5)  # adds vertical padding to cells
    for c in table.get_celld().values():
        c.visible_edges = 'horizontal'


fig, axs = plt.subplots(ncols=1, nrows=6, figsize=(7, 11))
seeing_ax, temp_ax, humidity_ax, pressure_ax, windspeed_ax, winddir_ax = axs

# Plot seeing
try:
    seeing_ax.plot(dimm_fwhm_df.index, dimm_fwhm_df.fwhm, label="Seeing (arcsec)")
    draw_summary_table(seeing_ax, dimm_fwhm_df, "fwhm", "Seeing", "arcsec")
except (AttributeError, KeyError):
    pass
seeing_ax.set_ylabel("Seeing (arcsec)")
conf_axes(seeing_ax, label_twilight=True)


# Plot air temperature
temp_ax.plot(ess_temp_df.index, ess_temp_df.temperatureItem0, label="Air temp.")
temp_ax.plot(ess_dewpoint_df.index, ess_dewpoint_df.dewPointItem, label="Dew point")
temp_ax.set_ylabel("Temperature (˚C)")
temp_ax.legend(loc='lower right')
conf_axes(temp_ax)
draw_summary_table(temp_ax, ess_temp_df, "temperatureItem0", "Air temp.", "˚C")

# Plot humidity
humidity_ax.plot(ess_humidity_df.index, ess_humidity_df.relativeHumidityItem)
humidity_ax.set_ylabel("Humidity (%)")
conf_axes(humidity_ax)
draw_summary_table(humidity_ax, ess_humidity_df, "relativeHumidityItem", "Humidity", "%")

# Plot atmospheric pressure
pressure_ax.plot(ess_pressure_df.index, ess_pressure_df.pressureItem0 / 100.)
pressure_ax.set_ylabel("Pressure (mbar)")
conf_axes(pressure_ax)
draw_summary_table(pressure_ax, ess_pressure_df / 100., "pressureItem0", "Atm. Pressure", "mbar")

# Plot wind speed
windspeed_ax.plot(ess_wind_df.index, ess_wind_df.speed)
windspeed_ax.set_ylabel("Wind (m/s)")
conf_axes(windspeed_ax)
draw_summary_table(windspeed_ax, ess_wind_df, "speed", "Wind speed", "m/s")

# Plot wind direction
winddir_ax.plot(ess_wind_df.index, ess_wind_df.direction)
winddir_ax.set_ylabel("Wind dir. (deg)")
winddir_ax.set_ylim(0., 360.)
conf_axes(winddir_ax, label_xaxis=True)

axs[-1].set_xlabel("Time (MM-DD HH, UTC)")
plt.setp(axs[-1].get_xticklabels(), rotation=30, ha="right")
plt.show()